# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 10 - Processamento de Linguagem Natural</font>

****** Este Jupyter Notebook foi atualizado para a versão 3.6.1. da Linguagem Python em 05/07/2017 ******

## Bag of Words


O modelo de "saco de palavras" é uma representação simplificada usada no processamento de linguagem natural e recuperação de informação. Neste modelo, um texto (como uma sentença ou um documento) é representado como o saco (multiset) de suas palavras, desconsiderando a gramática e até a ordem das palavras, mas mantendo a multiplicidade.

Na classificação de documentos, um saco de palavras é um vetor esparso de ocorrência de contagens de palavras; Ou seja, um histograma esparso sobre o vocabulário.

### Carregando um Dataset de um Site de E-commerce (em português)

In [1]:
import gzip
import json

In [2]:
# Carregando o dataset
corpus = list()
with gzip.open('ecommerce.json.gz') as fp:
    for line in fp:
        entry = line.decode('utf8')
        corpus.append(json.loads(entry))

In [3]:
from pprint import pprint
pprint(corpus[0])

{'_id': 120008322,
 'cat': ' Automotivo',
 'descr': 'Chegou o kit que junta resistência e conforto, além de níveis '
          'máximos de segurança. São 4 pneus para seu carro ficar completo e '
          'com a qualificação que você precisa.\n'
          'Com os conhecimentos avançados de hoje e um entusiasmo pela '
          'direção, os engenheiros da Pirelli puderam dar grandes passos. Cada '
          'pneu da Pirelli é responsável não só pelo desempenho, mas também '
          'por uma "vontade de ir pra estrada", comunicando-se com o motorista '
          'e gerando um melhor entendimento do desempenho do veículo, ou seja, '
          'a Pirelli transforma a sua viagem em uma aventura divertida e livre '
          'de problemas. Pneu Pirelli para carros com rodas aro 16, modelo '
          'high performance Phanthon, perfil baixo proporcionando maior '
          'estabilidade nas curvas, excelente qualidade e durabilidade para '
          'pistas.\n'
          '\n'
          'I

In [4]:
print (corpus[0]['descr'])

Chegou o kit que junta resistência e conforto, além de níveis máximos de segurança. São 4 pneus para seu carro ficar completo e com a qualificação que você precisa.
Com os conhecimentos avançados de hoje e um entusiasmo pela direção, os engenheiros da Pirelli puderam dar grandes passos. Cada pneu da Pirelli é responsável não só pelo desempenho, mas também por uma "vontade de ir pra estrada", comunicando-se com o motorista e gerando um melhor entendimento do desempenho do veículo, ou seja, a Pirelli transforma a sua viagem em uma aventura divertida e livre de problemas. Pneu Pirelli para carros com rodas aro 16, modelo high performance Phanthon, perfil baixo proporcionando maior estabilidade nas curvas, excelente qualidade e durabilidade para pistas.

Imagens meramente ilustrativas.
Todas as informações divulgadas são de responsabilidade do fabricante/fornecedor.


## Gensim - Modelagem de Tópicos

https://github.com/RaRe-Technologies/gensim

In [5]:
# Por enquanto disponíve, apenas para Python 2.7
#!pip install pattern

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
!pip install gensim

In [8]:
import gensim
print (gensim.summarization.summarize(corpus[0]['descr']))

Using TensorFlow backend.


São 4 pneus para seu carro ficar completo e com a qualificação que você precisa.


In [9]:
len(corpus)

65875

In [10]:
# Construindo um classificador para produtos e categorias (considerando apenas os 10 mil primeiros produtos)
dataset = list()
for entry in corpus[:10000]:
    if 'cat' in entry:
        dataset.append( (entry['name'], entry['cat'].lower().strip()) )

In [11]:
len(dataset)

9953

In [12]:
pprint(dataset[:10])

[('Kit com 4 Pneus de Alta Performance Pirelli Aro 16 205/55R16 Phantom',
  'automotivo'),
 ('Chandon Brut Rosé 750 ml', 'alimentos e bebidas'),
 ('Kit com 2 Vodkas Sueca Absolut Vanilia 1000ml', 'alimentos e bebidas'),
 ('Kit  - Livros de Colorir: Jardim Secreto + Floresta Encantada + Reino '
  'Animal',
  'livros'),
 ("Livro - Assassin's Creed: Submundo", 'livros'),
 ('BCAA 2400 - 100 Cápsulas - Nitech Nutrition', 'suplementos e vitaminas'),
 ('100% Whey - 900g - Baunilha - Nitech Nutrition', 'suplementos e vitaminas'),
 ('Whey Protein Isolate - 900g - Morango - Nitech Nutrition',
  'suplementos e vitaminas'),
 ('100% Whey - 900g - Chocolate - Nitech Nutrition', 'suplementos e vitaminas'),
 ('BCAA 2400 - 200 Cápsulas - Nitech Nutrition', 'suplementos e vitaminas')]


In [13]:
# Quantas categorias distintas nós temos e quantos itens por categoria?
from collections import Counter
counter = Counter([cat for prod, cat in dataset])
pprint(counter.most_common())

[('bebês', 1208),
 ('eletroportáteis', 1052),
 ('automotivo', 915),
 ('utilidades domésticas', 857),
 ('suplementos e vitaminas', 787),
 ('ar-condicionado e aquecedores', 754),
 ('informática', 706),
 ('cama, mesa e banho', 670),
 ('tv e home theater', 644),
 ('perfumaria', 532),
 ('beleza e saúde', 497),
 ('dvds e blu-ray', 433),
 ('relógios', 410),
 ('pet shop', 391),
 ('instrumentos musicais', 44),
 ('celulares e telefones', 18),
 ('eletrodomésticos', 16),
 ('áudio', 13),
 ('alimentos e bebidas', 2),
 ('livros', 2),
 ('brinquedos', 1),
 ('linha industrial', 1)]


# Construindo um Classificador SVM com Bag of Words

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')

In [16]:
# Construindo o modelo SVM com Pipeline
modelo = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel = 'linear', probability = True))])

In [17]:
?LabelEncoder

In [18]:
# Objeto para Normalização dos labels
encoder = LabelEncoder()

In [19]:
# Obtendo dados e labels
data = [prod for prod, cat in dataset]
labels = [cat for prod, cat in dataset]
len(data)

9953

In [20]:
# Normalização dos labels
target = encoder.fit_transform(labels)

In [21]:
# Items
encoder.classes_.item(1)

'ar-condicionado e aquecedores'

In [22]:
# Fit do modelo
modelo.fit(data, target)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [23]:
# Prevendo a categoria a partir da descrição
modelo.predict(["Refrigerador Brastemp com função frostfree"])

array([9])

In [24]:
# Prevendo a categoria a partir da descrição
print (encoder.classes_[9])

eletrodomésticos


In [25]:
# Probabilidades de um produto
probs = modelo.predict_proba(["Ventilador"])

In [26]:
# Probabidades de categorias para o objeto Ventilador
guess = [(class_, probs.item(n)) for n, class_ in enumerate(encoder.classes_)]
pprint(guess)

[('alimentos e bebidas', 3.0406646534438505e-07),
 ('ar-condicionado e aquecedores', 2.251189200106969e-07),
 ('automotivo', 2.102751716801002e-07),
 ('bebês', 4.0603364199716415e-07),
 ('beleza e saúde', 2.246598349887976e-07),
 ('brinquedos', 1.58788410972354e-05),
 ('cama, mesa e banho', 2.105293961752545e-07),
 ('celulares e telefones', 1.2556537440277665e-05),
 ('dvds e blu-ray', 2.566530048056465e-07),
 ('eletrodomésticos', 6.217194653236212e-07),
 ('eletroportáteis', 0.9999476284281454),
 ('informática', 2.571872970374585e-07),
 ('instrumentos musicais', 7.456165551172302e-07),
 ('linha industrial', 1.3529963706122157e-05),
 ('livros', 6.233002492617043e-07),
 ('perfumaria', 2.285690603247755e-07),
 ('pet shop', 3.5999830326413175e-07),
 ('relógios', 1.3231925039317137e-06),
 ('suplementos e vitaminas', 5.1842142000881e-07),
 ('tv e home theater', 3.2140321759791916e-07),
 ('utilidades domésticas', 2.006660574765312e-07),
 ('áudio', 3.3688190468262286e-06)]


In [27]:
# Probabidade ajustada de categorias para o objeto Ventilador
from operator import itemgetter
for cat, proba in sorted(guess, key = itemgetter(1), reverse = True):
    print ('{}: {:.4f}'.format(cat, proba))

eletroportáteis: 0.9999
brinquedos: 0.0000
linha industrial: 0.0000
celulares e telefones: 0.0000
áudio: 0.0000
relógios: 0.0000
instrumentos musicais: 0.0000
livros: 0.0000
eletrodomésticos: 0.0000
suplementos e vitaminas: 0.0000
bebês: 0.0000
pet shop: 0.0000
tv e home theater: 0.0000
alimentos e bebidas: 0.0000
informática: 0.0000
dvds e blu-ray: 0.0000
perfumaria: 0.0000
ar-condicionado e aquecedores: 0.0000
beleza e saúde: 0.0000
cama, mesa e banho: 0.0000
automotivo: 0.0000
utilidades domésticas: 0.0000


### Fim

### Obrigado - Data Science Academy - <a href=http://facebook.com/dsacademy>facebook.com/dsacademybr</a>